# Data analysis
## September 2025

* Step 1: import the data set
* Step 2: find the number of segmentations, and their size (number of pixels per slice, so we can get 3D volume)
* Step 3: Overlay the segmentations on the other channels
* Step 4: Get the intensity in each channel for the segmentations
* Step 5: Save it out



In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib

import matplotlib.pyplot as plt
from csbdeep.utils import Path

from glob import glob
from tqdm import tqdm
from tifffile import imread

import os

from skimage.measure import label, regionprops, regionprops_table


import numpy as np
import pandas as pd


### Step 1: Import image

In [2]:
Image_path = "3d_monolayer_xy1_ch2.tif"
mask_path = "output/StarDist/3d_demo3d_monolayer_xy1_ch2_labels.tif"
save_directory = "output/analysis"

In [3]:
segmentations = imread(mask_path)
intensity_image = imread(Image_path)

In [4]:
#If the voxels are anisotropic, change this here from 1's, to represent what they are
X_ani = 1
Y_ani = 1
Z_ani = 1

## Make some functions useful for our calculations

In [5]:

def count_pixels_for_label(segmentation_array, target_label):
    
    """
    Count the number of pixels for a specific label in each slice of a 3D segmentation array.
    Only include slices where the count of pixels for the target label is greater than zero.
    
    Parameters:
    - segmentation_array (np.ndarray): A 3D numpy array where each slice represents a segmentation map.
    - target_label (int): The label for which pixel counts are to be computed.
    
    Returns:
    - List[Tuple[int, int]]: A list of tuples where each tuple contains the slice number and the count of pixels for the target label.
    """
    
    # Initialize a list to store counts of the target label for each slice along with slice index
    pixel_counts = []

    # Iterate through each slice
    for slice_index in range(segmentation_array.shape[0]):
        slice_data = segmentation_array[slice_index]
        
        # Count the number of pixels for the target label in this slice
        count = np.sum(slice_data == target_label)
        
        # Only append to the list if count is greater than zero
        if count > 0:
            pixel_counts.append((slice_index, count))
    

    return pixel_counts

def volume_of_labels(segmentation_array, x_size=1, y_size=1, z_size=1):

    volume_dict={}
    total_volume_dict = {}
    for label in np.unique(segmentation_array):
        if label>0:
            total_volume=0
            volume_dict[label]=count_pixels_for_label(segmentation_array, label)
            for slice in volume_dict[label]:
                slice_volume = slice[1]*x_size*y_size*z_size
                total_volume = total_volume+slice_volume
            total_volume_dict[label]=total_volume
    

    return volume_dict, total_volume_dict

def get_z_slices(df, segmentation_array):
    total_slices = {}
    for target_label in df['label']:
        slice_count = np.sum(np.any(segmentation_array == target_label, axis=(1, 2)))
        total_slices[target_label]=slice_count

    return total_slices



### Rescale stardist mask
So that the stardist mask dimensions match the intensity image dimensions

In [6]:
import skimage.transform

# Resize segmentation mask to match intensity image shape using nearest-neighbour interpolation
segmentations_rescaled = skimage.transform.resize(
    segmentations,
    intensity_image.shape,
    order=0, # nearest-neihbour interpolation
    preserve_range=True, # preserve original range of values
).astype(segmentations.dtype)



### Measure properties of the intentisty image with the stardist-generated mask
Using region props

In [7]:
#Use region props to get the properties of the intensity image wherever there are labels in the label image
#The intensity image can be any of the channels from the image
data_dict =regionprops_table(segmentations_rescaled, intensity_image = intensity_image, properties=('label', 'area', 'slice', 'image_intensity', 'intensity_max', 'intensity_mean', 'intensity_min'))

In [8]:
data_dict['total_intensity'] = [sum(x[~np.isnan(x)]) if isinstance(x, np.ndarray) else 0 for x in data_dict['image_intensity']]   #sum the total intensity of the object across all slices 
total_slices = get_z_slices(data_dict, segmentations_rescaled) #get the number of slices for doing volume
total_slices = dict(sorted(total_slices.items()))
total_slices = [total_slices[key] for key in sorted(total_slices)]
data_dict.pop('image_intensity') #get rid of image intensity, as that's per slice, and we want the whole total image
data_dict.pop('slice') #get rid of slice number

volume_dict, total_volumes = volume_of_labels(segmentations_rescaled, X_ani, Y_ani, Z_ani) #calculate the Volume

data_frame = pd.DataFrame(data_dict) #make a dataframe
data_frame['total_volume'] = data_frame['label'].map(total_volumes)


In [9]:
data_frame

,label,area,intensity_max,intensity_mean,intensity_min,total_intensity,total_volume
0,1,19428.0,140.0,114.212477,101.0,2218920,19428
1,2,19396.0,152.0,113.250052,99.0,2196598,19396
2,3,13712.0,132.0,111.179186,100.0,1524489,13712
3,4,16692.0,133.0,111.673017,101.0,1864046,16692
4,5,14068.0,145.0,115.092550,101.0,1619122,14068
5,6,14996.0,136.0,111.339957,100.0,1669654,14996
6,7,16568.0,137.0,114.517262,100.0,1897322,16568
7,8,10528.0,162.0,118.799107,102.0,1250717,10528
8,9,10156.0,151.0,117.675857,101.0,1195116,10156
9,10,11840.0,155.0,116.279307,101.0,1376747,11840


In [10]:
Path(save_directory).mkdir(parents=True, exist_ok=True)
save_file = os.path.basename(Image_path)[:-4]+"_data.csv"
save_path= os.path.join(save_directory, save_file)

print(save_path)
data_frame.to_csv(save_path, index=False)

output/analysis\3d_monolayer_xy1_ch2_data.csv


In [11]:
# Save rescaled stardist mask
import skimage
skimage.io.imsave(save_directory+"_rescaled_image.tif", segmentations_rescaled)


C:\Users\dantuor\.conda\envs\calm-3dsegm\Lib\site-packages\skimage\_shared\utils.py:328: UserWarning: output/analysis_rescaled_image.tif is a low contrast image
  return func(*args, **kwargs)
